In [2]:
import scanpy as sc
import numpy as np 
from scanpy import AnnData
from tqdm import tqdm

In [30]:
# Xs = [sc.read_h5ad(f"../Data/h5ad/h5ad/plate{i}_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad", backed="r") for i in tqdm(range(1, 14 + 1))]
Xs = [sc.read_h5ad(f"data/h5ad/reduced/plate{i}.h5ad", backed="r") for i in tqdm(range(1, 14 + 1))]
# X = sc.read_h5ad("data/tahoe_sampled_1M.h5ad.gz")
# Xs = [X]
Xs

100%|██████████| 14/14 [01:49<00:00,  7.86s/it]


[AnnData object with n_obs × n_vars = 171297 × 62710 backed at 'data/h5ad/reduced/plate1.h5ad'
     obs: 'sample', 'gene_count', 'tscp_count', 'mread_count', 'drugname_drugconc', 'drug', 'cell_line', 'sublibrary', 'BARCODE', 'pcnt_mito', 'S_score', 'G2M_score', 'phase', 'pass_filter', 'cell_name', 'plate',
 AnnData object with n_obs × n_vars = 6892456 × 62710 backed at 'data/h5ad/reduced/plate2.h5ad'
     obs: 'sample', 'gene_count', 'tscp_count', 'mread_count', 'drugname_drugconc', 'drug', 'cell_line', 'sublibrary', 'BARCODE', 'pcnt_mito', 'S_score', 'G2M_score', 'phase', 'pass_filter', 'cell_name', 'plate',
 AnnData object with n_obs × n_vars = 1417313 × 62710 backed at 'data/h5ad/reduced/plate3.h5ad'
     obs: 'sample', 'gene_count', 'tscp_count', 'mread_count', 'drugname_drugconc', 'drug', 'cell_line', 'sublibrary', 'BARCODE', 'pcnt_mito', 'S_score', 'G2M_score', 'phase', 'pass_filter', 'cell_name', 'plate',
 AnnData object with n_obs × n_vars = 155688 × 62710 backed at 'data/h5ad/

In [31]:
# Define held out data for the validation and test sets
val_one_per_moa = ["Phenylephrine (hydrochloride)","Darolutamide", "palbociclib", "Tolmetin", "Procainamide (hydrochloride)", "Trifluridine", "Simotinib", "Methylprednisolone succinate", "Dapagliflozin", "CP21R7", "Panobinostat", "Tofacitinib", "Trametinib", "Vinblastine (sulfate)", "Temsirolimus", "Sunitinib", "Ralimetinib dimesylate", "Tirabrutinib", "GSK1059615", "SBI-0640756", "Lonafarnib", "Retinoic acid"]
val_one_entire_moa = ["Bortezomib", "Ixazomib", "Ixazomib citrate"]

val_one_per_organ = ["CVCL_1724", "CVCL_0179", "CVCL_1715", "CVCL_0366", "CVCL_1550", "CVCL_0480", "CVCL_0069" ]
val_one_entire_organ = ["CVCL_0359"]

test_one_per_moa = ["Vilanterol", "Flutamide", "Ribociclib", "Valdecoxib", "γ-Oryzanol", "Trimetrexate", "Tucatinib", "Triamcinolone", "Dapagliflozin ((2S)-1,2-propanediol, hydrate)", "LY2090314", "Tucidinostat", "Tofacitinib (citrate)", "Trametinib (DMSO_TF solvate)", "vincristine", "Torkinib", "Vandetanib", "Temuterkib", "Tirabrutinib (hydrochloride)", "Ipatasertib", "Tomivosertib", "RMC-6236", "Tazarotene"]
test_one_entire_moa = ["Glasdegib" , "Sonidegib", "Vismodegib"]

test_one_per_organ = ["CVCL_0397", "CVCL_1239", "CVCL_0371", "CVCL_1716", "CVCL_C466", "CVCL_1666", "CVCL_0293"]
test_one_entire_organ = ["CVCL_1094"]

val_plate_14 = lambda x:  x.obs_names[0::2]
test_plate_14 = lambda x : x.obs_names[1::2]

held_out_val = lambda x : x.obs["drug"].isin(val_one_per_moa) | x.obs["drug"].isin(val_one_entire_moa) | x.obs["cell_line"].isin(val_one_per_organ) | x.obs["cell_line"].isin(val_one_entire_organ) | ((x.obs["plate"] == "plate14") &  (x.obs_names.isin(val_plate_14(x))))
held_out_test = lambda x : x.obs["drug"].isin(test_one_per_moa) | x.obs["drug"].isin(test_one_entire_moa) | x.obs["cell_line"].isin(test_one_per_organ) | x.obs["cell_line"].isin(test_one_entire_organ)  | ((x.obs["plate"] == "plate14") &  (x.obs_names.isin(test_plate_14(x))))

In [32]:
import polars as pl
concentrations = pl.read_csv("concentrations.csv")

drug_to_concentration = {
    row[0]: row[1]
    for row in concentrations.iter_rows()
}

concentrations

drug,drugname_drugconc,len,dose
str,str,i64,str
"""(R)-Verapamil (hydrochloride)""","""[('(R)-Verapamil (hydrochlorid…",166943,""" 5.0"""
"""(S)-Crizotinib""","""[('(S)-Crizotinib', 0.5, 'uM')…",86840,""" 0.5"""
"""18β-Glycyrrhetinic acid""","""[('18β-Glycyrrhetinic acid', 5…",113159,""" 5.0"""
"""4EGI-1""","""[('4EGI-1', 0.5, 'uM')]""",128549,""" 0.5"""
"""5-Azacytidine""","""[('5-Azacytidine', 5.0, 'uM')]""",71466,""" 5.0"""
…,…,…,…
"""olaparib""","""[('olaparib', 0.5, 'uM')]""",136783,""" 0.5"""
"""palbociclib""","""[('palbociclib', 0.5, 'uM')]""",91681,""" 0.5"""
"""venetoclax""","""[('venetoclax', 0.5, 'uM')]""",118167,""" 0.5"""


In [33]:
for d in [val_one_per_moa, val_one_entire_moa, val_one_per_organ, val_one_entire_organ, test_one_per_moa, test_one_entire_moa, test_one_per_organ, test_one_entire_organ]:
    for i in d:
        count = sum([x[(x.obs["drug"] == i) | (x.obs["cell_line"] == i)].n_obs for x in Xs])
        # Assert to filter leads to zero cells
        if count == 0:
            print(f"{i} {count} leads to zero cells")


In [35]:
Xs = [X[(X.obs["pass_filter"] == "full") & (X.obs["drugname_drugconc"].astype(str) == X.obs["drug"].map(lambda x: drug_to_concentration[x]).astype(str))] for X in Xs]


KeyboardInterrupt: 

In [ ]:
# Xs[0].write_h5ad("data/tahoe_sampled_filtered.h5ad")

In [36]:
from typing import Callable
import pandas as pd

def random_subset(adatas: list[AnnData], scale : int = 1, filter : Callable[[AnnData], "pd.Series[bool]"] | None = None) -> AnnData:
    np.random.seed(42)
    if filter == None:
        filter = lambda x: pd.Series(True, index=x.obs.index)

    # Ys = [X[(X.obs["pass_filter"] == "full") & (X.obs["drugname_drugconc"].astype(str) == X.obs["drug"].map(lambda x: drug_to_concentration[x]).astype(str)) & (filter(X))] for X in adatas]
    Ys = [X[(filter(X))] for X in adatas]
    Rs = [np.random.choice(Y.obs_names, size=len(Y)//scale, replace=False) for Y in Ys] if scale != 1 else Ys
    xs = [X[r, :] for X, r in zip(adatas, Rs)]
    x = sc.concat([x.to_memory() for x in xs])
    return x


In [37]:
SCALES = [10000, 1000, 100, 10, 1]

In [38]:
for SCALE in SCALES:
    print(f"Creating train subset with scale {SCALE}")
    xs = random_subset(Xs, SCALE, lambda x: (~held_out_val(x)) & (~held_out_test(x)))
    xs.write_h5ad(f"../Data/subsets/train_{SCALE}x.h5ad")

    print(f"Creating val subset with scale {SCALE}")
    # First category of hold outs: Drugs/Cell lines where we have trained on other drugs/cell lines from the same MOA/organ
    xs = random_subset(Xs, SCALE, lambda x: x.obs["drug"].isin(val_one_per_moa))
    xs.write_h5ad(f"../Data/subsets/val_shared_MOA_{SCALE}x.h5ad")

    xs = random_subset(Xs, SCALE, lambda x: x.obs["cell_line"].isin(val_one_per_organ))
    xs.write_h5ad(f"../Data/subsets/val_shared_organ_{SCALE}x.h5ad")

    # Second category of hold outs: Drugs/Cell lines where we have not trained on any other drugs/cell lines from the same MOA/organ
    xs = random_subset(Xs, SCALE, lambda x: x.obs["drug"].isin(val_one_entire_moa))
    xs.write_h5ad(f"../Data/subsets/val_unseen_MOA_{SCALE}x.h5ad")

    xs = random_subset(Xs, SCALE, lambda x: x.obs["cell_line"].isin(val_one_entire_organ))
    xs.write_h5ad(f"../Data/subsets/val_unseen_organ_{SCALE}x.h5ad")

    # plate 14 
    xs = random_subset(Xs, SCALE, lambda x: ((x.obs["plate"] == "plate14") &  (x.obs_names.isin(val_plate_14(x)))))
    xs.write_h5ad(f"../Data/subsets/val_plate14_{SCALE}x.h5ad")

    # Now test data 
    print(f"Creating test subset with scale {SCALE}")

    xs = random_subset(Xs, SCALE, lambda x: x.obs["drug"].isin(test_one_per_moa))
    xs.write_h5ad(f"../Data/subsets/test_shared_MOA_{SCALE}x.h5ad")

    xs = random_subset(Xs, SCALE, lambda x: x.obs["cell_line"].isin(test_one_per_organ))
    xs.write_h5ad(f"../Data/subsets/test_shared_organ_{SCALE}x.h5ad")

    xs = random_subset(Xs, SCALE, lambda x: x.obs["drug"].isin(test_one_entire_moa))
    xs.write_h5ad(f"../Data/subsets/test_unseen_MOA_{SCALE}x.h5ad")

    xs = random_subset(Xs, SCALE, lambda x: x.obs["cell_line"].isin(test_one_entire_organ))
    xs.write_h5ad(f"../Data/subsets/test_unseen_organ_{SCALE}x.h5ad")

    xs = random_subset(Xs, SCALE, lambda x: ((x.obs["plate"] == "plate14") &  (x.obs_names.isin(test_plate_14(x)))))
    xs.write_h5ad(f"../Data/subsets/test_plate14_{SCALE}x.h5ad")


Creating train subset with scale 10000
Creating val subset with scale 10000
Creating test subset with scale 10000
Creating train subset with scale 1000
Creating val subset with scale 1000
Creating test subset with scale 1000
Creating train subset with scale 100
Creating val subset with scale 100
Creating test subset with scale 100
